## 导入相关包

In [1]:
import pandas as pd
import numpy as np

## 读取数据（训练数据前10000行，测试数据前100条）

In [2]:
train_data = pd.read_csv('train_all.csv',nrows=10000)
test_data = pd.read_csv('test_all.csv',nrows=100)

## 读取全部数据

In [3]:
# train_data = pd.read_csv('train_all.csv',nrows=None)
# test_data = pd.read_csv('test_all.csv',nrows=None)

## 获取训练和测试数据

In [5]:
features_columns = [col for col in train_data.columns if col not in ['user_id','label']]
train = train_data[features_columns].values
test = test_data[features_columns].values
target =train_data['label'].values

## 切分40%数据用于线下验证

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0, n_jobs=-1)
X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.4, random_state=0)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)  

(1200, 229) (1200,)
(800, 229) (800,)


0.9275

## 交叉验证：评估估算器性能

In [8]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0, n_jobs=-1)
scores = cross_val_score(clf, train, target, cv=5)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2)) 

[0.9375 0.935  0.935  0.935  0.935 ]
Accuracy: 0.94 (+/- 0.00)


## F1验证

In [9]:
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0, n_jobs=-1)
scores = cross_val_score(clf, train, target, cv=5, scoring='f1_macro')
print(scores)  
print("F1: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[0.48387097 0.48320413 0.48320413 0.48320413 0.48320413]
F1: 0.48 (+/- 0.00)


## ShuffleSplit切分数据

In [12]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0, n_jobs=-1)
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
cross_val_score(clf, train, target, cv=cv)  

array([0.93333333, 0.943     , 0.93133333, 0.93833333, 0.934     ])

## 自己写交叉验证

### KFlod切分数据

In [10]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0, n_jobs=-1)
kf = KFold(n_splits=5)
for k, (train_index, test_index) in enumerate(kf.split(train)):
    X_train, X_test, y_train, y_test = train[train_index], train[test_index], target[train_index], target[test_index]
    clf = clf.fit(X_train, y_train)
    print(k, clf.score(X_test, y_test))

0 0.9375
1 0.925
2 0.935
3 0.9325
4 0.9475


### StratifiedKFold切分数据(label均分)

In [11]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0, n_jobs=-1)
skf = StratifiedKFold(n_splits=5)
for k, (train_index, test_index) in enumerate(skf.split(train, target)):
    X_train, X_test, y_train, y_test = train[train_index], train[test_index], target[train_index], target[test_index]
    clf = clf.fit(X_train, y_train)
    print(k, clf.score(X_test, y_test))

0 0.9375
1 0.935
2 0.935
3 0.935
4 0.935


## 模型调参

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier


# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.5, random_state=0)

# model 
clf = RandomForestClassifier(n_jobs=-1)

# Set the parameters by cross-validation

tuned_parameters = {
                    'n_estimators': [50, 100, 200]
#                     ,'criterion': ['gini', 'entropy']
#                     ,'max_depth': [2, 5]
#                     ,'max_features': ['log2', 'sqrt', 'int']
#                     ,'bootstrap': [True, False]
#                     ,'warm_start': [True, False]
                    }

scores = ['precision']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(clf, tuned_parameters, cv=5,
                       scoring='%s_macro' % score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'n_estimators': 100}

Grid scores on development set:

0.622 (+/-0.400) for {'n_estimators': 50}
0.639 (+/-0.423) for {'n_estimators': 100}
0.622 (+/-0.400) for {'n_estimators': 200}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

              precision    recall  f1-score   support

         0.0       0.93      0.98      0.95       928
         1.0       0.00      0.00      0.00        72

    accuracy                           0.91      1000
   macro avg       0.46      0.49      0.48      1000
weighted avg       0.86      0.91      0.89      1000




## 模糊矩阵

In [13]:
import itertools
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier

# label name
class_names = ['no-repeat', 'repeat']

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(train, target, random_state=0)

# Run classifier, using a model that is too regularized (C too low) to see
# the impact on the results
clf = RandomForestClassifier(n_jobs=-1)
y_pred = clf.fit(X_train, y_train).predict(X_test)


def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()


# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

Confusion matrix, without normalization
[[460   7]
 [ 32   1]]
Normalized confusion matrix
[[0.99 0.01]
 [0.97 0.03]]


<Figure size 640x480 with 2 Axes>

<Figure size 640x480 with 2 Axes>

In [14]:
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

# label name
class_names = ['no-repeat', 'repeat']

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(train, target, random_state=0)

# Run classifier, using a model that is too regularized (C too low) to see
# the impact on the results
clf = RandomForestClassifier(n_jobs=-1)
y_pred = clf.fit(X_train, y_train).predict(X_test)

print(classification_report(y_test, y_pred, target_names=class_names))

              precision    recall  f1-score   support

   no-repeat       0.93      0.99      0.96       467
      repeat       0.12      0.03      0.05        33

    accuracy                           0.92       500
   macro avg       0.53      0.51      0.50       500
weighted avg       0.88      0.92      0.90       500



## 不同的分类模型

### LR 模型

In [17]:
train_data.head()

,user_id,merchant_id,age_range,gender,user_cnt,seller_nunique,cat_nunique,brand_nunique,item_nunique,time_stamp_nunique,...,embeeding_93,embeeding_94,embeeding_95,embeeding_96,embeeding_97,embeeding_98,embeeding_99,lgb_clf,xgb_clf,label
0,34176,3906.0,6.0,0.0,451.0,109.0,45.0,106.0,256.0,47.0,...,-0.014218,0.026865,-0.533239,0.231678,-0.427409,0.412302,0.850330,0.928806,0.833814,0.0
1,34176,121.0,6.0,0.0,451.0,109.0,45.0,106.0,256.0,47.0,...,-0.014218,0.026865,-0.533239,0.231678,-0.427409,0.412302,0.850330,0.921832,0.879359,0.0
2,34176,4356.0,6.0,0.0,451.0,109.0,45.0,106.0,256.0,47.0,...,-0.014218,0.026865,-0.533239,0.231678,-0.427409,0.412302,0.850330,0.940439,0.930820,1.0
3,34176,2217.0,6.0,0.0,451.0,109.0,45.0,106.0,256.0,47.0,...,-0.014218,0.026865,-0.533239,0.231678,-0.427409,0.412302,0.850330,0.928806,0.834748,0.0
4,230784,4818.0,0.0,0.0,54.0,20.0,17.0,19.0,31.0,16.0,...,-0.128846,-0.287453,-0.151249,0.713761,-0.563686,-0.011947,0.135482,0.940592,0.929739,0.0


In [19]:
train_data.age_range.unique(),train_data.gender.unique()

(array([6., 0., 4., 5., 3., 2., 7., 8.]), array([0., 1., 2.]))

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

stdScaler = StandardScaler()
X = stdScaler.fit_transform(train)

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, target, random_state=0)

clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(X_train, y_train)
clf.score(X_test, y_test)

e:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.9

### KNN 模型

In [22]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

stdScaler = StandardScaler()
X = stdScaler.fit_transform(train)

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, target, random_state=0)

clf = KNeighborsClassifier(n_neighbors=3).fit(X_train, y_train)
clf.score(X_test, y_test)

0.926

## GaussianNB 模型

In [23]:
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler

stdScaler = StandardScaler()
X = stdScaler.fit_transform(train)

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, target, random_state=0)

clf = GaussianNB().fit(X_train, y_train)
clf.score(X_test, y_test)

0.436

## tree树模型

In [24]:
from sklearn import tree

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(train, target, random_state=0)

clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.836

## bagging模型

In [25]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(train, target, random_state=0)
clf = BaggingClassifier(KNeighborsClassifier(), max_samples=0.5, max_features=0.5)

clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.934

## 随机森林模型

In [26]:
from sklearn.ensemble import RandomForestClassifier

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(train, target, random_state=0)
clf = clf = RandomForestClassifier(n_estimators=10, max_depth=None, min_samples_split=2, random_state=0)

clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.924

## ExTree模型

In [27]:
from sklearn.ensemble import ExtraTreesClassifier

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(train, target, random_state=0)
clf = ExtraTreesClassifier(n_estimators=10, max_depth=None, min_samples_split=2, random_state=0)

clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.922

## AdaBoost模型

In [28]:
from sklearn.ensemble import AdaBoostClassifier

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(train, target, random_state=0)
clf = AdaBoostClassifier(n_estimators=100)

clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.928

## GBDT模型

In [29]:
from sklearn.ensemble import GradientBoostingClassifier

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(train, target, random_state=0)
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)

clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.926

## VOTE模型投票

In [30]:
from sklearn import datasets
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import StandardScaler

stdScaler = StandardScaler()
X = stdScaler.fit_transform(train)
y = target


clf1 = LogisticRegression(solver='lbfgs', multi_class='multinomial', random_state=1)
clf2 = RandomForestClassifier(n_estimators=50, random_state=1)
clf3 = GaussianNB()

eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='hard')

for clf, label in zip([clf1, clf2, clf3, eclf], ['Logistic Regression', 'Random Forest', 'naive Bayes', 'Ensemble']):
    scores = cross_val_score(clf, X, y, cv=5, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

e:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
e:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_

Accuracy: 0.89 (+/- 0.01) [Logistic Regression]
Accuracy: 0.93 (+/- 0.00) [Random Forest]
Accuracy: 0.47 (+/- 0.01) [naive Bayes]


e:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
e:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_

Accuracy: 0.90 (+/- 0.01) [Ensemble]


## lgb 模型

In [41]:
import lightgbm

X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.4, random_state=0)
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size=0.5, random_state=0)

clf = lightgbm

train_matrix = clf.Dataset(X_train, label=y_train)
test_matrix = clf.Dataset(X_test, label=y_test)
params = {
          'boosting_type': 'gbdt',
          #'boosting_type': 'dart',
          'objective': 'multiclass',
          'metric': 'multi_logloss',
          'min_child_weight': 1.5,
          'num_leaves': 2**5,
          'lambda_l2': 10,
          'subsample': 0.7,
          'colsample_bytree': 0.7,
          'colsample_bylevel': 0.7,
          'learning_rate': 0.03,
          'tree_method': 'exact',
          'seed': 2017,
          "num_class": 2,
          'silent': True,
          }
num_round = 10000
early_stopping_rounds = 100
model = clf.train(params, 
                  train_matrix,
                  num_round,
                  valid_sets=test_matrix,
                  early_stopping_rounds=early_stopping_rounds)
pre= model.predict(X_valid,num_iteration=model.best_iteration)

e:\anaconda3\lib\site-packages\lightgbm\basic.py:842: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  .format(key))


[1]	valid_0's multi_logloss: 0.317263
Training until validation scores don't improve for 100 rounds
[2]	valid_0's multi_logloss: 0.317184
[3]	valid_0's multi_logloss: 0.317602
[4]	valid_0's multi_logloss: 0.317859
[5]	valid_0's multi_logloss: 0.318005
[6]	valid_0's multi_logloss: 0.317868
[7]	valid_0's multi_logloss: 0.31785
[8]	valid_0's multi_logloss: 0.318022
[9]	valid_0's multi_logloss: 0.318117
[10]	valid_0's multi_logloss: 0.318356
[11]	valid_0's multi_logloss: 0.318413
[12]	valid_0's multi_logloss: 0.318844
[13]	valid_0's multi_logloss: 0.319306
[14]	valid_0's multi_logloss: 0.319465
[15]	valid_0's multi_logloss: 0.319577
[16]	valid_0's multi_logloss: 0.319914
[17]	valid_0's multi_logloss: 0.319986
[18]	valid_0's multi_logloss: 0.320511
[19]	valid_0's multi_logloss: 0.320969
[20]	valid_0's multi_logloss: 0.321219
[21]	valid_0's multi_logloss: 0.321337
[22]	valid_0's multi_logloss: 0.321495
[23]	valid_0's multi_logloss: 0.321668
[24]	valid_0's multi_logloss: 0.321823
[25]	valid_0

In [32]:
print('score : ', np.mean((pre[:,1]>0.5)==y_valid))

score :  0.9475


## xgb 模型

In [46]:
import xgboost

X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.4, random_state=0)
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size=0.5, random_state=0)

clf = xgboost

train_matrix = clf.DMatrix(X_train, label=y_train, missing=-1)
test_matrix = clf.DMatrix(X_test, label=y_test, missing=-1)
z = clf.DMatrix(X_valid, label=y_valid, missing=-1)
params = {'booster': 'gbtree',
          'objective': 'multi:softprob',
          'eval_metric': 'mlogloss',
          'gamma': 1,
          'min_child_weight': 1.5,
          'max_depth': 5,
          'lambda': 10,
          'subsample': 0.7,
          'colsample_bytree': 0.7,
          'colsample_bylevel': 0.7,
          'eta': 0.03,
          'tree_method': 'exact',
          'seed': 2017,
          "num_class": 2
          }

num_round = 10000
early_stopping_rounds = 100
watchlist = [(train_matrix, 'train'),
             (test_matrix, 'eval')
             ]

model = clf.train(params,
                  train_matrix,
                  num_boost_round=num_round,
                  evals=watchlist,
                  early_stopping_rounds=early_stopping_rounds
                  )
pre = model.predict(z,ntree_limit=model.best_ntree_limit)

[0]	train-mlogloss:0.67031	eval-mlogloss:0.67265
Multiple eval metrics have been passed: 'eval-mlogloss' will be used for early stopping.

Will train until eval-mlogloss hasn't improved in 100 rounds.
[1]	train-mlogloss:0.64879	eval-mlogloss:0.65312
[2]	train-mlogloss:0.62844	eval-mlogloss:0.63498
[3]	train-mlogloss:0.60916	eval-mlogloss:0.61784
[4]	train-mlogloss:0.59086	eval-mlogloss:0.60183
[5]	train-mlogloss:0.57368	eval-mlogloss:0.58669
[6]	train-mlogloss:0.55731	eval-mlogloss:0.57199
[7]	train-mlogloss:0.54182	eval-mlogloss:0.55845
[8]	train-mlogloss:0.52705	eval-mlogloss:0.54562
[9]	train-mlogloss:0.51293	eval-mlogloss:0.53343
[10]	train-mlogloss:0.49945	eval-mlogloss:0.52189
[11]	train-mlogloss:0.48654	eval-mlogloss:0.51070
[12]	train-mlogloss:0.47422	eval-mlogloss:0.50013
[13]	train-mlogloss:0.46269	eval-mlogloss:0.49014
[14]	train-mlogloss:0.45145	eval-mlogloss:0.48065
[15]	train-mlogloss:0.44092	eval-mlogloss:0.47199
[16]	train-mlogloss:0.43060	eval-mlogloss:0.46339
[17]	tra

[160]	train-mlogloss:0.12293	eval-mlogloss:0.32637
[161]	train-mlogloss:0.12250	eval-mlogloss:0.32659
[162]	train-mlogloss:0.12193	eval-mlogloss:0.32677
[163]	train-mlogloss:0.12143	eval-mlogloss:0.32699
[164]	train-mlogloss:0.12089	eval-mlogloss:0.32710
[165]	train-mlogloss:0.12036	eval-mlogloss:0.32743
[166]	train-mlogloss:0.11989	eval-mlogloss:0.32788
[167]	train-mlogloss:0.11931	eval-mlogloss:0.32834
[168]	train-mlogloss:0.11888	eval-mlogloss:0.32879
[169]	train-mlogloss:0.11838	eval-mlogloss:0.32892
[170]	train-mlogloss:0.11793	eval-mlogloss:0.32911
[171]	train-mlogloss:0.11761	eval-mlogloss:0.32947
[172]	train-mlogloss:0.11726	eval-mlogloss:0.32965
[173]	train-mlogloss:0.11683	eval-mlogloss:0.32959
[174]	train-mlogloss:0.11640	eval-mlogloss:0.32982
[175]	train-mlogloss:0.11612	eval-mlogloss:0.33011
[176]	train-mlogloss:0.11567	eval-mlogloss:0.33041
[177]	train-mlogloss:0.11525	eval-mlogloss:0.33045
[178]	train-mlogloss:0.11467	eval-mlogloss:0.33045
[179]	train-mlogloss:0.11426	ev

In [65]:
print('score : ', np.mean((pre[:,1]>0.5)==y_valid))

score :  0.9435


# 自己封装模型

## Stacking,Bootstrap,Bagging技术实践

In [51]:
"""
    导入相关包
"""
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

In [52]:
class SBBTree():
    """
        SBBTree
        Stacking,Bootstap,Bagging
    """
    def __init__(
                    self, 
                    params,
                    stacking_num,
                    bagging_num,
                    bagging_test_size,
                    num_boost_round,
                    early_stopping_rounds
                ):
        """
            Initializes the SBBTree.
            Args:
              params : lgb params.
              stacking_num : k_flod stacking.
              bagging_num : bootstrap num.
              bagging_test_size : bootstrap sample rate.
              num_boost_round : boost num.
              early_stopping_rounds : early_stopping_rounds.
        """
        self.params = params
        self.stacking_num = stacking_num
        self.bagging_num = bagging_num
        self.bagging_test_size = bagging_test_size
        self.num_boost_round = num_boost_round
        self.early_stopping_rounds = early_stopping_rounds

        self.model = lgb
        self.stacking_model = []
        self.bagging_model = []

    def fit(self, X, y):
        """ fit model. """
        if self.stacking_num > 1:
            layer_train = np.zeros((X.shape[0], 2))
            self.SK = StratifiedKFold(n_splits=self.stacking_num, shuffle=True, random_state=1)
            for k,(train_index, test_index) in enumerate(self.SK.split(X, y)):
                X_train = X[train_index]
                y_train = y[train_index]
                X_test = X[test_index]
                y_test = y[test_index]

                lgb_train = lgb.Dataset(X_train, y_train)
                lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

                gbm = lgb.train(self.params,
                            lgb_train,
                            num_boost_round=self.num_boost_round,
                            valid_sets=lgb_eval,
                            early_stopping_rounds=self.early_stopping_rounds)

                self.stacking_model.append(gbm)

                pred_y = gbm.predict(X_test, num_iteration=gbm.best_iteration)
                layer_train[test_index, 1] = pred_y

            X = np.hstack((X, layer_train[:,1].reshape((-1,1)))) 
        else:
            pass
        for bn in range(self.bagging_num):
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=self.bagging_test_size, random_state=bn)

            lgb_train = lgb.Dataset(X_train, y_train)
            lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

            gbm = lgb.train(self.params,
                        lgb_train,
                        num_boost_round=10000,
                        valid_sets=lgb_eval,
                        early_stopping_rounds=200)

            self.bagging_model.append(gbm)

    def predict(self, X_pred):
        """ predict test data. """
        if self.stacking_num > 1:
            test_pred = np.zeros((X_pred.shape[0], self.stacking_num))
            for sn,gbm in enumerate(self.stacking_model):
                pred = gbm.predict(X_pred, num_iteration=gbm.best_iteration)
                test_pred[:, sn] = pred
            X_pred = np.hstack((X_pred, test_pred.mean(axis=1).reshape((-1,1))))  
        else:
            pass 
        for bn,gbm in enumerate(self.bagging_model):
            pred = gbm.predict(X_pred, num_iteration=gbm.best_iteration)
            if bn == 0:
                pred_out=pred
            else:
                pred_out+=pred
        return pred_out/self.bagging_num

## 测试自己封装的模型类

In [53]:
"""
    TEST CODE
"""
from sklearn.datasets import make_classification
from sklearn.datasets import load_breast_cancer
from sklearn.datasets import make_gaussian_quantiles
from sklearn import metrics
from sklearn.metrics import f1_score
# X, y = make_classification(n_samples=1000, n_features=25, n_clusters_per_class=1, n_informative=15, random_state=1)
X, y = make_gaussian_quantiles(mean=None, cov=1.0, n_samples=1000, n_features=50, n_classes=2, shuffle=True, random_state=2)
# data = load_breast_cancer()
# X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'num_leaves': 9,
        'learning_rate': 0.03,
        'feature_fraction_seed': 2,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'min_data': 20,
        'min_hessian': 1,
        'verbose': -1,
        'silent': 0
        }
# test 1
model = SBBTree(params=params, stacking_num=2, bagging_num=1,  bagging_test_size=0.33, num_boost_round=10000, early_stopping_rounds=200)
model.fit(X,y)
X_pred = X[0].reshape((1,-1))
pred=model.predict(X_pred)
print('pred')
print(pred)
print('TEST 1 ok')


# test 1
model = SBBTree(params, stacking_num=1, bagging_num=1, bagging_test_size=0.33, num_boost_round=10000, early_stopping_rounds=200)
model.fit(X_train,y_train)
pred1=model.predict(X_test)

# test 2 
model = SBBTree(params, stacking_num=1, bagging_num=3, bagging_test_size=0.33, num_boost_round=10000, early_stopping_rounds=200)
model.fit(X_train,y_train)
pred2=model.predict(X_test)

# test 3 
model = SBBTree(params, stacking_num=5, bagging_num=1, bagging_test_size=0.33, num_boost_round=10000, early_stopping_rounds=200)
model.fit(X_train,y_train)
pred3=model.predict(X_test)

# test 4 
model = SBBTree(params, stacking_num=5, bagging_num=3, bagging_test_size=0.33, num_boost_round=10000, early_stopping_rounds=200)
model.fit(X_train,y_train)
pred4=model.predict(X_test)

fpr, tpr, thresholds = metrics.roc_curve(y_test+1, pred1, pos_label=2)
print('auc: ',metrics.auc(fpr, tpr))

fpr, tpr, thresholds = metrics.roc_curve(y_test+1, pred2, pos_label=2)
print('auc: ',metrics.auc(fpr, tpr))

fpr, tpr, thresholds = metrics.roc_curve(y_test+1, pred3, pos_label=2)
print('auc: ',metrics.auc(fpr, tpr))

fpr, tpr, thresholds = metrics.roc_curve(y_test+1, pred4, pos_label=2)
print('auc: ',metrics.auc(fpr, tpr))

e:\anaconda3\lib\site-packages\lightgbm\basic.py:842: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  .format(key))


[1]	valid_0's auc: 0.589432
Training until validation scores don't improve for 200 rounds
[2]	valid_0's auc: 0.59292
[3]	valid_0's auc: 0.59092
[4]	valid_0's auc: 0.61048
[5]	valid_0's auc: 0.611936
[6]	valid_0's auc: 0.610368
[7]	valid_0's auc: 0.611112
[8]	valid_0's auc: 0.613168
[9]	valid_0's auc: 0.6156
[10]	valid_0's auc: 0.620088
[11]	valid_0's auc: 0.624968
[12]	valid_0's auc: 0.625864
[13]	valid_0's auc: 0.620952
[14]	valid_0's auc: 0.622024
[15]	valid_0's auc: 0.616968
[16]	valid_0's auc: 0.615464
[17]	valid_0's auc: 0.614152
[18]	valid_0's auc: 0.611928
[19]	valid_0's auc: 0.61068
[20]	valid_0's auc: 0.614656
[21]	valid_0's auc: 0.617568
[22]	valid_0's auc: 0.622752
[23]	valid_0's auc: 0.624368
[24]	valid_0's auc: 0.630704
[25]	valid_0's auc: 0.633136
[26]	valid_0's auc: 0.632496
[27]	valid_0's auc: 0.631648
[28]	valid_0's auc: 0.634224
[29]	valid_0's auc: 0.636512
[30]	valid_0's auc: 0.635392
[31]	valid_0's auc: 0.63744
[32]	valid_0's auc: 0.639216
[33]	valid_0's auc: 0.6389

[349]	valid_0's auc: 0.733184
[350]	valid_0's auc: 0.732976
[351]	valid_0's auc: 0.733024
[352]	valid_0's auc: 0.733536
[353]	valid_0's auc: 0.733824
[354]	valid_0's auc: 0.734224
[355]	valid_0's auc: 0.73496
[356]	valid_0's auc: 0.73536
[357]	valid_0's auc: 0.735664
[358]	valid_0's auc: 0.735568
[359]	valid_0's auc: 0.736256
[360]	valid_0's auc: 0.736064
[361]	valid_0's auc: 0.736688
[362]	valid_0's auc: 0.736768
[363]	valid_0's auc: 0.737456
[364]	valid_0's auc: 0.737536
[365]	valid_0's auc: 0.737936
[366]	valid_0's auc: 0.737152
[367]	valid_0's auc: 0.736304
[368]	valid_0's auc: 0.736288
[369]	valid_0's auc: 0.735072
[370]	valid_0's auc: 0.734912
[371]	valid_0's auc: 0.735568
[372]	valid_0's auc: 0.736368
[373]	valid_0's auc: 0.736752
[374]	valid_0's auc: 0.73736
[375]	valid_0's auc: 0.737888
[376]	valid_0's auc: 0.737296
[377]	valid_0's auc: 0.737616
[378]	valid_0's auc: 0.73744
[379]	valid_0's auc: 0.737472
[380]	valid_0's auc: 0.737296
[381]	valid_0's auc: 0.738288
[382]	valid_0'

[798]	valid_0's auc: 0.74704
[799]	valid_0's auc: 0.746768
[800]	valid_0's auc: 0.746784
[801]	valid_0's auc: 0.74664
[802]	valid_0's auc: 0.746576
[803]	valid_0's auc: 0.746496
[804]	valid_0's auc: 0.746656
[805]	valid_0's auc: 0.746672
[806]	valid_0's auc: 0.746528
[807]	valid_0's auc: 0.746352
[808]	valid_0's auc: 0.745968
[809]	valid_0's auc: 0.746064
[810]	valid_0's auc: 0.746016
[811]	valid_0's auc: 0.745616
[812]	valid_0's auc: 0.745264
[813]	valid_0's auc: 0.745248
[814]	valid_0's auc: 0.745296
[815]	valid_0's auc: 0.745136
[816]	valid_0's auc: 0.745216
[817]	valid_0's auc: 0.745344
[818]	valid_0's auc: 0.74536
[819]	valid_0's auc: 0.745152
[820]	valid_0's auc: 0.745152
[821]	valid_0's auc: 0.745376
[822]	valid_0's auc: 0.745216
[823]	valid_0's auc: 0.745152
[824]	valid_0's auc: 0.745104
[825]	valid_0's auc: 0.744768
[826]	valid_0's auc: 0.744752
[827]	valid_0's auc: 0.744672
[828]	valid_0's auc: 0.744448
[829]	valid_0's auc: 0.74472
[830]	valid_0's auc: 0.744448
[831]	valid_0'

[233]	valid_0's auc: 0.75896
[234]	valid_0's auc: 0.759424
[235]	valid_0's auc: 0.76032
[236]	valid_0's auc: 0.760112
[237]	valid_0's auc: 0.759264
[238]	valid_0's auc: 0.759808
[239]	valid_0's auc: 0.759216
[240]	valid_0's auc: 0.759168
[241]	valid_0's auc: 0.760176
[242]	valid_0's auc: 0.760416
[243]	valid_0's auc: 0.760928
[244]	valid_0's auc: 0.760864
[245]	valid_0's auc: 0.760864
[246]	valid_0's auc: 0.761488
[247]	valid_0's auc: 0.761984
[248]	valid_0's auc: 0.761888
[249]	valid_0's auc: 0.762432
[250]	valid_0's auc: 0.762384
[251]	valid_0's auc: 0.763648
[252]	valid_0's auc: 0.764736
[253]	valid_0's auc: 0.764992
[254]	valid_0's auc: 0.765712
[255]	valid_0's auc: 0.766576
[256]	valid_0's auc: 0.767248
[257]	valid_0's auc: 0.766832
[258]	valid_0's auc: 0.766384
[259]	valid_0's auc: 0.766976
[260]	valid_0's auc: 0.766944
[261]	valid_0's auc: 0.767152
[262]	valid_0's auc: 0.76696
[263]	valid_0's auc: 0.766448
[264]	valid_0's auc: 0.766384
[265]	valid_0's auc: 0.766048
[266]	valid_0

...

[534]	valid_0's auc: 0.782308
[535]	valid_0's auc: 0.782308
[536]	valid_0's auc: 0.782531
[537]	valid_0's auc: 0.782754
[538]	valid_0's auc: 0.782308
[539]	valid_0's auc: 0.782977
[540]	valid_0's auc: 0.7832
[541]	valid_0's auc: 0.782086
[542]	valid_0's auc: 0.782086
[543]	valid_0's auc: 0.781863
[544]	valid_0's auc: 0.780749
[545]	valid_0's auc: 0.780526
[546]	valid_0's auc: 0.780749
[547]	valid_0's auc: 0.781417
[548]	valid_0's auc: 0.781194
[549]	valid_0's auc: 0.782086
[550]	valid_0's auc: 0.780971
[551]	valid_0's auc: 0.780303
[552]	valid_0's auc: 0.781194
[553]	valid_0's auc: 0.780526
[554]	valid_0's auc: 0.78008
[555]	valid_0's auc: 0.780303
[556]	valid_0's auc: 0.779857
[557]	valid_0's auc: 0.778743
[558]	valid_0's auc: 0.779412
[559]	valid_0's auc: 0.779635
[560]	valid_0's auc: 0.779189
[561]	valid_0's auc: 0.778966
[562]	valid_0's auc: 0.778966
[563]	valid_0's auc: 0.778298
[564]	valid_0's auc: 0.778298
[565]	valid_0's auc: 0.778966
[566]	valid_0's auc: 0.781194
[567]	valid_0

[230]	valid_0's auc: 0.747165
[231]	valid_0's auc: 0.747083
[232]	valid_0's auc: 0.747246
[233]	valid_0's auc: 0.747328
[234]	valid_0's auc: 0.748144
[235]	valid_0's auc: 0.74847
[236]	valid_0's auc: 0.749123
[237]	valid_0's auc: 0.749286
[238]	valid_0's auc: 0.749694
[239]	valid_0's auc: 0.749694
[240]	valid_0's auc: 0.749939
[241]	valid_0's auc: 0.750591
[242]	valid_0's auc: 0.750999
[243]	valid_0's auc: 0.750184
[244]	valid_0's auc: 0.750347
[245]	valid_0's auc: 0.750347
[246]	valid_0's auc: 0.751081
[247]	valid_0's auc: 0.750918
[248]	valid_0's auc: 0.750347
[249]	valid_0's auc: 0.751326
[250]	valid_0's auc: 0.751081
[251]	valid_0's auc: 0.751489
[252]	valid_0's auc: 0.751407
[253]	valid_0's auc: 0.751163
[254]	valid_0's auc: 0.750591
[255]	valid_0's auc: 0.750184
Early stopping, best iteration is:
[55]	valid_0's auc: 0.756139
[1]	valid_0's auc: 0.749451
Training until validation scores don't improve for 200 rounds
[2]	valid_0's auc: 0.739293
[3]	valid_0's auc: 0.738033
[4]	valid_0

[358]	valid_0's auc: 0.766843
[359]	valid_0's auc: 0.766436
[360]	valid_0's auc: 0.766761
[361]	valid_0's auc: 0.766843
[362]	valid_0's auc: 0.766843
[363]	valid_0's auc: 0.76668
[364]	valid_0's auc: 0.766436
[365]	valid_0's auc: 0.766436
[366]	valid_0's auc: 0.767087
[367]	valid_0's auc: 0.766843
[368]	valid_0's auc: 0.767087
[369]	valid_0's auc: 0.766924
[370]	valid_0's auc: 0.767087
[371]	valid_0's auc: 0.765868
[372]	valid_0's auc: 0.765949
[373]	valid_0's auc: 0.765217
[374]	valid_0's auc: 0.76473
[375]	valid_0's auc: 0.764405
[376]	valid_0's auc: 0.764892
[377]	valid_0's auc: 0.765461
[378]	valid_0's auc: 0.764892
[379]	valid_0's auc: 0.764323
[380]	valid_0's auc: 0.764405
[381]	valid_0's auc: 0.764161
[382]	valid_0's auc: 0.764323
[383]	valid_0's auc: 0.764323
[384]	valid_0's auc: 0.764974
[385]	valid_0's auc: 0.76473
[386]	valid_0's auc: 0.764486
[387]	valid_0's auc: 0.764242
[388]	valid_0's auc: 0.763998
[389]	valid_0's auc: 0.763104
[390]	valid_0's auc: 0.763673
[391]	valid_0

[132]	valid_0's auc: 0.778355
[133]	valid_0's auc: 0.779167
[134]	valid_0's auc: 0.779248
[135]	valid_0's auc: 0.780791
[136]	valid_0's auc: 0.779654
[137]	valid_0's auc: 0.779492
[138]	valid_0's auc: 0.779248
[139]	valid_0's auc: 0.780547
[140]	valid_0's auc: 0.779248
[141]	valid_0's auc: 0.779817
[142]	valid_0's auc: 0.777949
[143]	valid_0's auc: 0.778517
[144]	valid_0's auc: 0.778274
[145]	valid_0's auc: 0.777543
[146]	valid_0's auc: 0.779005
[147]	valid_0's auc: 0.778842
[148]	valid_0's auc: 0.779248
[149]	valid_0's auc: 0.780547
[150]	valid_0's auc: 0.78006
[151]	valid_0's auc: 0.779817
[152]	valid_0's auc: 0.780141
[153]	valid_0's auc: 0.780953
[154]	valid_0's auc: 0.780791
[155]	valid_0's auc: 0.780791
[156]	valid_0's auc: 0.780872
[157]	valid_0's auc: 0.780466
[158]	valid_0's auc: 0.780872
[159]	valid_0's auc: 0.780628
[160]	valid_0's auc: 0.780466
[161]	valid_0's auc: 0.782252
[162]	valid_0's auc: 0.781359
[163]	valid_0's auc: 0.782577
[164]	valid_0's auc: 0.782577
[165]	valid

[503]	valid_0's auc: 0.785662
[504]	valid_0's auc: 0.785824
[505]	valid_0's auc: 0.786068
[506]	valid_0's auc: 0.785337
[507]	valid_0's auc: 0.78485
[508]	valid_0's auc: 0.784282
[509]	valid_0's auc: 0.783714
[510]	valid_0's auc: 0.783226
[511]	valid_0's auc: 0.782902
[512]	valid_0's auc: 0.782658
[513]	valid_0's auc: 0.782902
[514]	valid_0's auc: 0.783308
[515]	valid_0's auc: 0.782983
[516]	valid_0's auc: 0.782739
[517]	valid_0's auc: 0.78282
[518]	valid_0's auc: 0.782252
[519]	valid_0's auc: 0.783551
[520]	valid_0's auc: 0.78347
[521]	valid_0's auc: 0.783632
[522]	valid_0's auc: 0.784038
[523]	valid_0's auc: 0.783795
[524]	valid_0's auc: 0.784363
[525]	valid_0's auc: 0.78347
[526]	valid_0's auc: 0.784038
[527]	valid_0's auc: 0.784444
[528]	valid_0's auc: 0.78412
[529]	valid_0's auc: 0.784444
[530]	valid_0's auc: 0.78485
[531]	valid_0's auc: 0.784688
[532]	valid_0's auc: 0.784688
[533]	valid_0's auc: 0.784688
[534]	valid_0's auc: 0.784525
[535]	valid_0's auc: 0.785013
[536]	valid_0's 

## 天猫复购场景实战

## 读取特征数据

In [54]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

train_data = pd.read_csv('train_all.csv',nrows=10000)
test_data = pd.read_csv('test_all.csv',nrows=100)

features_columns = [col for col in train_data.columns if col not in ['user_id','label']]
train = train_data[features_columns].values
test = test_data[features_columns].values
target =train_data['label'].values

## 设置模型参数

In [55]:
params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'num_leaves': 9,
        'learning_rate': 0.03,
        'feature_fraction_seed': 2,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'min_data': 20,
        'min_hessian': 1,
        'verbose': -1,
        'silent': 0
        }

model = SBBTree(params=params,
                stacking_num=5,
                bagging_num=3,
                bagging_test_size=0.33,
                num_boost_round=10000,
                early_stopping_rounds=200)

## 模型训练

In [56]:
model.fit(train, target)

e:\anaconda3\lib\site-packages\lightgbm\basic.py:842: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  .format(key))


[1]	valid_0's auc: 0.463733
Training until validation scores don't improve for 200 rounds
[2]	valid_0's auc: 0.456267
[3]	valid_0's auc: 0.5256
[4]	valid_0's auc: 0.561653
[5]	valid_0's auc: 0.55344
[6]	valid_0's auc: 0.536907
[7]	valid_0's auc: 0.51184
[8]	valid_0's auc: 0.496533
[9]	valid_0's auc: 0.49392
[10]	valid_0's auc: 0.504427
[11]	valid_0's auc: 0.483893
[12]	valid_0's auc: 0.48144
[13]	valid_0's auc: 0.47344
[14]	valid_0's auc: 0.469387
[15]	valid_0's auc: 0.47232
[16]	valid_0's auc: 0.486133
[17]	valid_0's auc: 0.490827
[18]	valid_0's auc: 0.501493
[19]	valid_0's auc: 0.502987
[20]	valid_0's auc: 0.504427
[21]	valid_0's auc: 0.48976
[22]	valid_0's auc: 0.490187
[23]	valid_0's auc: 0.48752
[24]	valid_0's auc: 0.490933
[25]	valid_0's auc: 0.493813
[26]	valid_0's auc: 0.492907
[27]	valid_0's auc: 0.493813
[28]	valid_0's auc: 0.49616
[29]	valid_0's auc: 0.49632
[30]	valid_0's auc: 0.493013
[31]	valid_0's auc: 0.49616
[32]	valid_0's auc: 0.49536
[33]	valid_0's auc: 0.49136
[34]	

[91]	valid_0's auc: 0.456191
[92]	valid_0's auc: 0.456911
[93]	valid_0's auc: 0.455368
[94]	valid_0's auc: 0.453106
[95]	valid_0's auc: 0.453003
[96]	valid_0's auc: 0.451872
[97]	valid_0's auc: 0.44745
[98]	valid_0's auc: 0.447964
[99]	valid_0's auc: 0.449506
[100]	valid_0's auc: 0.447141
[101]	valid_0's auc: 0.446421
[102]	valid_0's auc: 0.441896
[103]	valid_0's auc: 0.442822
[104]	valid_0's auc: 0.445496
[105]	valid_0's auc: 0.444159
[106]	valid_0's auc: 0.442308
[107]	valid_0's auc: 0.442205
[108]	valid_0's auc: 0.441176
[109]	valid_0's auc: 0.440251
[110]	valid_0's auc: 0.437783
[111]	valid_0's auc: 0.439737
[112]	valid_0's auc: 0.443336
[113]	valid_0's auc: 0.448478
[114]	valid_0's auc: 0.449918
[115]	valid_0's auc: 0.452489
[116]	valid_0's auc: 0.450123
[117]	valid_0's auc: 0.449815
[118]	valid_0's auc: 0.44745
[119]	valid_0's auc: 0.451563
[120]	valid_0's auc: 0.453928
[121]	valid_0's auc: 0.449918
[122]	valid_0's auc: 0.444673
[123]	valid_0's auc: 0.442616
[124]	valid_0's auc: 

[185]	valid_0's auc: 0.524476
[186]	valid_0's auc: 0.527252
[187]	valid_0's auc: 0.528692
[188]	valid_0's auc: 0.527664
[189]	valid_0's auc: 0.527149
[190]	valid_0's auc: 0.528281
[191]	valid_0's auc: 0.527664
[192]	valid_0's auc: 0.527972
[193]	valid_0's auc: 0.531263
[194]	valid_0's auc: 0.529412
[195]	valid_0's auc: 0.528795
[196]	valid_0's auc: 0.528281
[197]	valid_0's auc: 0.528795
[198]	valid_0's auc: 0.529
[199]	valid_0's auc: 0.529617
[200]	valid_0's auc: 0.52972
[201]	valid_0's auc: 0.527869
[202]	valid_0's auc: 0.530954
[203]	valid_0's auc: 0.528898
[204]	valid_0's auc: 0.529823
[205]	valid_0's auc: 0.531057
[206]	valid_0's auc: 0.530852
[207]	valid_0's auc: 0.530029
[208]	valid_0's auc: 0.527972
[209]	valid_0's auc: 0.528178
[210]	valid_0's auc: 0.526429
[211]	valid_0's auc: 0.523961
[212]	valid_0's auc: 0.523241
[213]	valid_0's auc: 0.522624
[214]	valid_0's auc: 0.523241
[215]	valid_0's auc: 0.52355
[216]	valid_0's auc: 0.526738
[217]	valid_0's auc: 0.528795
[218]	valid_0's

[74]	valid_0's auc: 0.50833
[75]	valid_0's auc: 0.505656
[76]	valid_0's auc: 0.509461
[77]	valid_0's auc: 0.510901
[78]	valid_0's auc: 0.512855
[79]	valid_0's auc: 0.511312
[80]	valid_0's auc: 0.514397
[81]	valid_0's auc: 0.516865
[82]	valid_0's auc: 0.518717
[83]	valid_0's auc: 0.520156
[84]	valid_0's auc: 0.52355
[85]	valid_0's auc: 0.520773
[86]	valid_0's auc: 0.518614
[87]	valid_0's auc: 0.517894
[88]	valid_0's auc: 0.519745
[89]	valid_0's auc: 0.524476
[90]	valid_0's auc: 0.52211
[91]	valid_0's auc: 0.520259
[92]	valid_0's auc: 0.523344
[93]	valid_0's auc: 0.52355
[94]	valid_0's auc: 0.519436
[95]	valid_0's auc: 0.522933
[96]	valid_0's auc: 0.524167
[97]	valid_0's auc: 0.523756
[98]	valid_0's auc: 0.528383
[99]	valid_0's auc: 0.533114
[100]	valid_0's auc: 0.533628
[101]	valid_0's auc: 0.538564
[102]	valid_0's auc: 0.538359
[103]	valid_0's auc: 0.541135
[104]	valid_0's auc: 0.545557
[105]	valid_0's auc: 0.545249
[106]	valid_0's auc: 0.544838
[107]	valid_0's auc: 0.543706
[108]	vali

[167]	valid_0's auc: 0.562492
[168]	valid_0's auc: 0.56259
[169]	valid_0's auc: 0.560557
[170]	valid_0's auc: 0.557639
[171]	valid_0's auc: 0.559803
[172]	valid_0's auc: 0.559902
[173]	valid_0's auc: 0.562295
[174]	valid_0's auc: 0.563016
[175]	valid_0's auc: 0.564689
[176]	valid_0's auc: 0.565672
[177]	valid_0's auc: 0.568033
[178]	valid_0's auc: 0.56941
[179]	valid_0's auc: 0.56918
[180]	valid_0's auc: 0.57318
[181]	valid_0's auc: 0.572623
[182]	valid_0's auc: 0.571574
[183]	valid_0's auc: 0.57377
[184]	valid_0's auc: 0.572689
[185]	valid_0's auc: 0.571574
[186]	valid_0's auc: 0.572
[187]	valid_0's auc: 0.572525
[188]	valid_0's auc: 0.57423
[189]	valid_0's auc: 0.575344
[190]	valid_0's auc: 0.574361
[191]	valid_0's auc: 0.577443
[192]	valid_0's auc: 0.579148
[193]	valid_0's auc: 0.578623
[194]	valid_0's auc: 0.577934
[195]	valid_0's auc: 0.579574
[196]	valid_0's auc: 0.579344
[197]	valid_0's auc: 0.579508
[198]	valid_0's auc: 0.579508
[199]	valid_0's auc: 0.578557
[200]	valid_0's auc

[457]	valid_0's auc: 0.583738
[458]	valid_0's auc: 0.583377
[459]	valid_0's auc: 0.584033
[460]	valid_0's auc: 0.583836
[461]	valid_0's auc: 0.583869
[462]	valid_0's auc: 0.582951
[463]	valid_0's auc: 0.582525
[464]	valid_0's auc: 0.582754
[465]	valid_0's auc: 0.582295
[466]	valid_0's auc: 0.58177
[467]	valid_0's auc: 0.581508
[468]	valid_0's auc: 0.58082
[469]	valid_0's auc: 0.580361
[470]	valid_0's auc: 0.580295
[471]	valid_0's auc: 0.580164
[472]	valid_0's auc: 0.580197
[473]	valid_0's auc: 0.580164
[474]	valid_0's auc: 0.579738
[475]	valid_0's auc: 0.580459
[476]	valid_0's auc: 0.579443
[477]	valid_0's auc: 0.579279
[478]	valid_0's auc: 0.578918
[479]	valid_0's auc: 0.579541
[480]	valid_0's auc: 0.578918
[481]	valid_0's auc: 0.579082
[482]	valid_0's auc: 0.579443
[483]	valid_0's auc: 0.579115
[484]	valid_0's auc: 0.579279
[485]	valid_0's auc: 0.578066
[486]	valid_0's auc: 0.577508
[487]	valid_0's auc: 0.577377
[488]	valid_0's auc: 0.576557
[489]	valid_0's auc: 0.576951
[490]	valid_

[195]	valid_0's auc: 0.521767
[196]	valid_0's auc: 0.524368
[197]	valid_0's auc: 0.523237
[198]	valid_0's auc: 0.522559
[199]	valid_0's auc: 0.522747
[200]	valid_0's auc: 0.520975
[201]	valid_0's auc: 0.521955
[202]	valid_0's auc: 0.521579
Early stopping, best iteration is:
[2]	valid_0's auc: 0.548038
[1]	valid_0's auc: 0.593117
Training until validation scores don't improve for 200 rounds
[2]	valid_0's auc: 0.595021
[3]	valid_0's auc: 0.600335
[4]	valid_0's auc: 0.587803
[5]	valid_0's auc: 0.563624
[6]	valid_0's auc: 0.599808
[7]	valid_0's auc: 0.591798
[8]	valid_0's auc: 0.596321
[9]	valid_0's auc: 0.611945
[10]	valid_0's auc: 0.608929
[11]	valid_0's auc: 0.608232
[12]	valid_0's auc: 0.594908
[13]	valid_0's auc: 0.603709
[14]	valid_0's auc: 0.604745
[15]	valid_0's auc: 0.61937
[16]	valid_0's auc: 0.610663
[17]	valid_0's auc: 0.602484
[18]	valid_0's auc: 0.597358
[19]	valid_0's auc: 0.594606
[20]	valid_0's auc: 0.591384
[21]	valid_0's auc: 0.595869
[22]	valid_0's auc: 0.597245
[23]	va

## 预测结果

In [57]:
pred = model.predict(test)
df_out = pd.DataFrame()
df_out['user_id'] = test_data['user_id']
df_out['predict_prob'] = pred
df_out.head()

,user_id,predict_prob
0,NaN,0.041774
1,NaN,0.041760
2,NaN,0.041778
3,NaN,0.041757
4,NaN,0.047263


## 保存结果

In [59]:
"""
    保留数据头，不保存index
"""
df_out.to_csv('df_out.csv',header=True,index=False)
print('save OK!')

save OK!
